In [63]:
import requests
import geopandas as gpd
import geojson
import pandas as pd
from keys import mapbox_access_token


In [5]:
ames_housing = pd.read_csv('data/AmesHousing.csv')
realestate_data = pd.read_csv('data/Ames_Real_Estate_Data.csv')

neighbourhoods = pd.read_csv('data/neighbourhood_codes.csv', names=['codes', 'Neighborhood_name'])
neighbourhoods['codes']=neighbourhoods['codes'].str.strip()


C:\Users\local_x964246D\Temp\ipykernel_39120\812275049.py:2: DtypeWarning: Columns (36,40,81,87) have mixed types. Specify dtype option on import or set low_memory=False.
  realestate_data = pd.read_csv('data/Ames_Real_Estate_Data.csv')


In [6]:
cleaned_data = ames_housing.merge(neighbourhoods, left_on='Neighborhood', right_on='codes')

In [7]:
# Confirming no problems with the join
cleaned_data.loc[:,['Neighborhood', 'Neighborhood_name']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2487 entries, 0 to 2486
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Neighborhood       2487 non-null   object
 1   Neighborhood_name  2487 non-null   object
dtypes: object(2)
memory usage: 39.0+ KB


In [8]:
cleaned_data = cleaned_data.merge(realestate_data, left_on='PID', right_on='MapRefNo')

In [9]:
# Confirming no problems with the join
print(len(cleaned_data.loc[:,'PID'].unique()))
cleaned_data.loc[:,['PID', 'MapRefNo']]

2459


,PID,MapRefNo
0,527105010,527105010
1,527105030,527105030
2,527162130,527162130
3,527163010,527163010
4,527165230,527165230
...,...,...
2502,533350020,533350020
2503,532476050,532476050
2504,533250130,533250130
2505,533252020,533252020


In [15]:
import geocoder

# Aquiring json coordinates
cleaned_data['geolocation'] = cleaned_data['Prop_Addr_Full'].apply(lambda x: geocoder.arcgis(x).json)

In [25]:
for i in range(len(cleaned_data['geolocation'])):
    cleaned_data.loc[i, 'lat'] = cleaned_data['geolocation'][i]['lat']
    cleaned_data.loc[i, 'lng'] = cleaned_data['geolocation'][i]['lng']


In [52]:
# Remove entry with no address
cleaned_data = cleaned_data[~(cleaned_data['PID']==533210020)]

cleaned_data = cleaned_data.assign(price_sq2=cleaned_data['SalePrice_x']/cleaned_data['Lot Area'])

cleaned_data.to_csv('data/geodata.csv')

In [54]:
cleaned_data = cleaned_data.assign(geometry=gpd.points_from_xy(cleaned_data.lng, cleaned_data.lat, crs="EPSG:4326"))

In [55]:
cleaned_data_gdf = gpd.GeoDataFrame(cleaned_data, geometry='geometry')

In [56]:
display(cleaned_data.drop(columns='geometry'))

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,Prop_Addr_Full,geolocation,lat,lng,price_sq2
0,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,"4025 BERKSHIRE AVE, Ames, IA, USA","{'address': '4025 Berkshire Ave, Ames, Iowa, 5...",42.061218,-93.639232,13.731020
1,6,527105030,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,"4011 BERKSHIRE AVE, Ames, IA, USA","{'address': '4011 Berkshire Ave, Ames, Iowa, 5...",42.060868,-93.639124,19.593105
2,10,527162130,60,RL,60.0,7500,Pave,NaN,Reg,Lvl,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,"3814 BERKSHIRE AVE, Ames, IA, USA","{'address': '3814 Berkshire Ave, Ames, Iowa, 5...",42.058745,-93.638861,25.200000
3,11,527163010,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,"3835 BROOKDALE AVE, Ames, IA, USA","{'address': '3835 Brookdale Ave, Ames, Iowa, 5...",42.059264,-93.636950,17.590000
4,12,527165230,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,"3720 COLUMBINE CIR, Ames, IA, USA","{'address': '3720 Columbine Cir, Ames, Iowa, 5...",42.057716,-93.635441,23.182957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502,1861,533350020,20,RL,94.0,17778,Pave,NaN,IR1,Lvl,...,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,"3128 KINGMAN RD, Ames, IA, USA","{'address': '3128 Kingman Rd, Ames, Iowa, 5001...",42.036369,-93.658760,20.980988
2503,2499,532476050,80,RL,100.0,14330,Pave,NaN,IR1,Low,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,"1601 AMHERST DR, Ames, IA, USA","{'address': '1601 Amherst Dr, Ames, Iowa, 5001...",42.037406,-93.660672,18.143754
2504,2517,533250130,120,RL,30.0,9549,Pave,NaN,IR1,Lvl,...,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,"2877 GREENSBORO CIR, Ames, IA, USA","{'address': '2877 Greensboro Cir, Ames, Iowa, ...",42.044817,-93.649233,28.275212
2505,2518,533252020,20,RL,NaN,11250,Pave,NaN,IR1,Lvl,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,"2824 GREENSBORO DR, Ames, IA, USA","{'address': '2824 Greensboro Dr, Ames, Iowa, 5...",42.043848,-93.647509,14.666667


In [57]:
cleaned_data_gdf.dissolve().centroid

C:\Users\local_x964246D\Temp\ipykernel_39120\2975272298.py:1: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




0    POINT (-93.64773 42.03327)
dtype: geometry

In [58]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt

selected_column='Neighborhood_name'
gdf = cleaned_data_gdf
# Get all unique values from the 'category_column'
gdf = gdf.sort_values(by=selected_column)
unique_values = gdf.loc[:,selected_column].unique()

# Generate a list of distinct colors using the 'tab20' colormap from matplotlib
colors = plt.cm.tab20.colors
color_scale = [f'rgb({int(r * 255)},{int(g * 255)},{int(b * 255)})' for r, g, b in colors]
# Create the 'category_colors' dictionary by mapping each unique category to a color
unique_values_colours = {category: color for category, color in zip(unique_values, color_scale)}

# Map the categories in the GeoDataFrame to the corresponding colors
gdf['color'] = gdf[selected_column].map(unique_values_colours)

fig = go.Figure()

for val in unique_values:
    df_color = gdf[gdf[selected_column] == val]

    fig.add_trace(
        go.Scattermapbox(
                        name=val,
                        lat=df_color['lat'],
                        lon=df_color['lng'],
                        mode = 'markers',
                        marker = dict(
                            size = 5,
                            opacity = 1,

                            color = df_color['color'],
                        ),
                        # hovertext=df_color['tooltip'],
                        hovertemplate='<extra></extra>',
                    )
                )

fig.update_layout(
   autosize=True,
   hovermode='closest',
   mapbox=dict(
       accesstoken=mapbox_access_token,
       bearing=0,
       center=dict(
           lat=42.03327,
           lon=-93.64773
       ),
       pitch=0,
       zoom=10
   ),
)

display(fig)

In [59]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt

selected_column='price_sq2'
gdf = cleaned_data

gdf.dropna(subset=[selected_column], inplace=True)

fig = go.Figure()

fig.add_trace(
    go.Scattermapbox(
                    name=val,
                    lat=gdf['lat'],
                    lon=gdf['lng'],
                    mode = 'markers',
                    marker = dict(
                        size = 5,
                        opacity = 1,
                        color=gdf[selected_column],
                        colorscale='RdYlGn',
                        colorbar=dict(
                            title="Price per sq feet",
                            thicknessmode="pixels", thickness=50,
                            # lenmode="pixels", len=200,
                            yanchor="top", y=1,
                            ticks="outside",
                            dtick=5
                        )
                    ),
                    hovertext=gdf[selected_column],
                    # hovertemplate='<extra></extra>',
                )
            )

fig.update_layout(
   autosize=True,
   hovermode='closest',
   mapbox=dict(
       accesstoken=mapbox_access_token,
       bearing=0,
       center=dict(
           lat=42.03327,
           lon=-93.64773
       ),
       pitch=0,
       zoom=10
   ),
)

display(fig)